In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@78a0dfe2

In [2]:
import spark.implicits._

case class Employee(id: Int, name: String, salary: Int, departmentId: Int)
case class Department(id: Int, name: String)

val employeeData: Seq[(Int, String, Int, Int)] = Seq(
  (1, "Joe", 85000, 1),
  (2, "Henry", 80000, 2),
  (3, "Sam", 60000, 2),
  (4, "Max", 90000, 1),
  (5, "Janet", 69000, 1),
  (6, "Randy", 85000, 1),
  (7, "Will", 70000, 1)
)

val departmentData: Seq[(Int, String)] = Seq(
  (1, "IT"),
  (2, "Sales")
)

val employeeDS = employeeData.toDF("id", "name", "salary", "departmentId").as[Employee]
val departmentDS = departmentData.toDF("id", "name").as[Department]

import spark.implicits._


defined class Employee
defined class Department
employeeData: Seq[(Int, String, Int, Int)] = List(
  (1, "Joe", 85000, 1),
  (2, "Henry", 80000, 2),
  (3, "Sam", 60000, 2),
  (4, "Max", 90000, 1),
  (5, "Janet", 69000, 1),
  (6, "Randy", 85000, 1),
  (7, "Will", 70000, 1)
)
departmentData: Seq[(Int, String)] = List((1, "IT"), (2, "Sales"))
employeeDS: Dataset[Employee] = [id: int, name: string ... 2 more fields]
departmentDS: Dataset[Department] = [id: int, name: string]

A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.

Write a solution to find the employees who are high earners in each of the departments.

Return the result table in any order.

The result format is in the following example.

| Department | Employee | Salary |
|------------|----------|--------|
| IT         | Max      | 90000  |
| IT         | Joe      | 85000  |
| IT         | Randy    | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |

In [3]:
val windowSpec = Window.partitionBy(col("departmentId"))
    .orderBy(col("salary").desc)
    
employeeDS.withColumn("rank", dense_rank().over(windowSpec))
    .filter(col("rank") <= 3)
    .join(departmentDS, employeeDS("departmentId") === departmentDS("id"), "left_outer")
    .select(
        departmentDS("id").as("Department"),
        employeeDS("name").as("Name"),
        employeeDS("salary").as("Salary")
    )
    .show()

+----------+-----+------+
|Department| Name|Salary|
+----------+-----+------+
|         1|  Max| 90000|
|         1|  Joe| 85000|
|         1|Randy| 85000|
|         1| Will| 70000|
|         2|Henry| 80000|
|         2|  Sam| 60000|
+----------+-----+------+



windowSpec: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@45b378e9

In [4]:
spark.stop()